<a href="https://colab.research.google.com/github/zshujon/keras/blob/master/02.convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
ls /root

In [46]:
dir = 'drive/MyDrive/datasets/dog_vs_cat/images/'

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import random

import glob
import os
import itertools
import shutil

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix

In [9]:
_gpu = tf.config.experimental.list_physical_devices('GPU')
print(f'Number of GPU: {_gpu}')
if _gpu:
    tf.config.experimental.set_memory_growth(_gpu[0], True)

Number of GPU: []


In [53]:
os.chdir(dir)

FileNotFoundError: ignored

In [54]:
pwd

'/root'